In [128]:
#参考: https://qiita.com/toshiyuki_tsutsui/items/19590b464f15f845efcd

import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import MeCab

tagger = MeCab.Tagger('-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')#タグはMeCab.Tagger（neologd辞書）を使用

tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]



#学習データの読み込み

df = pd.read_csv('./csv/tokyo_ramen_top20_100.csv')
df_ramen = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)

#コーパス作成
wakati_ramen_text = []
for i in df_ramen['review']:
    txt = tokenize(i, 2, 10000, True)
    wakati_ramen_text.append(txt)

np.savetxt("./work/ramen_corpus.txt", wakati_ramen_text,fmt='%s', delimiter=',')
# np.savetxt("./work/ramen_corpus.txt", wakati_ramen_text, fmt = '%s', delimiter = ',')

# モデル作成

word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text,sg=1,vector_size=100,window=20,min_count=5,epochs=5,workers=3)
# word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text, sg = 1, vector_size = 100, window = 5, min_count = 5, epochs = 100, workers = 3)

#sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）
# size->vector_size，iter->epochsに変更

# モデルのセーブ
word2vec_ramen_model.save("./model/word2vec_ramen_model.model")

/Users/naoki_ito/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [129]:
from gensim.models import word2vec

word2vec_ramen_model = word2vec.Word2Vec.load("./model/word2vec_ramen_model.model")

#self.wv.most_similarを使う

#word2vec_ramen_model.wv.most_similar('とみ田') 


word2vec_ramen_model.wv.most_similar(positive=["かわいい"])

[('まわり', 0.9089914560317993),
 ('よ～', 0.8961716294288635),
 ('躊躇', 0.887725830078125),
 ('みな', 0.8863531351089478),
 ('い方', 0.8816729784011841),
 ('おら', 0.8811513185501099),
 ('小さく', 0.8807997703552246),
 ('正しい', 0.8794723749160767),
 ('おまけ', 0.8732083439826965),
 ('神保町', 0.8723004460334778)]